In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind, ttest_rel, mannwhitneyu
import func4behav as fb
import matplotlib
import imp
imp.reload(fb)

<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tACS-Attention\\processing\\behavior\\func4behav.py'>

In [5]:
def make_tables(case):
# case = 'endo valid'
    behavior_compare, experiment = fb.create_allsubs_compare()
    for subject_id in range (1,19):
        behavior_before, behavior_after = fb.load_behavior(subject_id)
        behavior_compare = fb.allsubs_compare(subject_id, behavior_before, behavior_after, behavior_compare, experiment, verbose=False)

    behavior_compare = behavior_compare.loc[(behavior_compare['response'] == 1) & 
                                            (behavior_compare['reaction time'] > 0.05) & (behavior_compare['reaction time'] < 1)]
    behavior_before, behavior_after = fb.filter_behav(case, behavior_compare.loc[behavior_compare['session'] == 'before'], 
                                                    behavior_compare.loc[behavior_compare['session'] == 'after'])

    behavior_compare = pd.concat([behavior_before, behavior_after])
    rt_sham_before = behavior_before.loc[behavior_compare['Real stimulation'] == 0]
    rt_sham_after = behavior_after.loc[behavior_compare['Real stimulation'] == 0 ]
    rt_real_before = behavior_before.loc[behavior_compare['Real stimulation'] == 1]
    rt_real_after = behavior_after.loc[behavior_compare['Real stimulation'] == 1]

    # preprocessing
    if case == 'endo invalid':
        rt_sham_before = fb.remove_outlier(rt_sham_before, k=1, left=False, right=True, verbose=False)
        rt_sham_after = fb.remove_outlier(rt_sham_after, k=0.6, left=True, right=False, verbose=False)
        rt_real_before = fb.remove_outlier(rt_real_before, k=1, left=True, right=False, verbose=False)
        rt_real_after = fb.remove_outlier(rt_real_after, k=0.8, left=False, right=True, verbose=False)

        rt_sham_before = rt_sham_before.loc[:, 'reaction time'].tolist()
        rt_sham_after = rt_sham_after.loc[:, 'reaction time'].tolist()
        rt_real_before = rt_real_before.loc[:, 'reaction time'].tolist()
        rt_real_after = rt_real_after.loc[:, 'reaction time'].tolist()

        # duplicates
        n_times = 4
        rt_sham_before = rt_sham_before * n_times
        rt_sham_after = rt_sham_after * n_times
        rt_real_before = rt_real_before * n_times
        rt_real_after = rt_real_after * n_times
    else:
        k_out = [1, 1, 1, 1]
        rt_sham_before = fb.remove_outlier(rt_sham_before, k=k_out[0], left=False, right=True, verbose=False)
        rt_sham_after = fb.remove_outlier(rt_sham_after, k=k_out[1], left=True, right=False, verbose=False)
        rt_real_before = fb.remove_outlier(rt_real_before, k=k_out[2], left=True, right=False, verbose=False)
        rt_real_after = fb.remove_outlier(rt_real_after, k=k_out[3], left=False, right=True, verbose=False)

        rt_sham_before = rt_sham_before.loc[:, 'reaction time'].tolist()
        rt_sham_after = rt_sham_after.loc[:, 'reaction time'].tolist()
        rt_real_before = rt_real_before.loc[:, 'reaction time'].tolist()
        rt_real_after = rt_real_after.loc[:, 'reaction time'].tolist()
        
    rt_sham_before = [num * 1000 for num in rt_sham_before]
    rt_sham_after = [num * 1000 for num in rt_sham_after]
    rt_real_before = [num * 1000 for num in rt_real_before]
    rt_real_after = [num * 1000 for num in rt_real_after]

    # make table
    df = pd.DataFrame(columns=['Group', 'PrePost', 'Type', 'RT'])
    # Create a new DataFrame for the new rows to be inserted
    new_rows = pd.DataFrame({
        'Group': ['sham'] * len(rt_sham_before),    # Repeat 'sham' for the length of rt_sham_before
        'PrePost': ['pre'] * len(rt_sham_before),   # Repeat 'pre' for the same length
        'Type': [case] * len(rt_sham_before),  # Repeat 'endo valid' for the same length
        'RT': rt_sham_before                        # Reaction times
    })
    df = pd.concat([df, new_rows], ignore_index=True)

    new_rows = pd.DataFrame({
        'Group': ['sham'] * len(rt_sham_after),    # Repeat 'sham' for the length of rt_sham_before
        'PrePost': ['post'] * len(rt_sham_after),   # Repeat 'pre' for the same length
        'Type': [case] * len(rt_sham_after),  # Repeat 'endo valid' for the same length
        'RT': rt_sham_after                        # Reaction times
    })
    df = pd.concat([df, new_rows], ignore_index=True)

    new_rows = pd.DataFrame({
        'Group': ['active'] * len(rt_real_before),    # Repeat 'sham' for the length of rt_sham_before
        'PrePost': ['pre'] * len(rt_real_before),   # Repeat 'pre' for the same length
        'Type': [case] * len(rt_real_before),  # Repeat 'endo valid' for the same length
        'RT': rt_real_before                        # Reaction times
    })
    df = pd.concat([df, new_rows], ignore_index=True)

    new_rows = pd.DataFrame({
        'Group': ['active'] * len(rt_real_after),    # Repeat 'sham' for the length of rt_sham_before
        'PrePost': ['post'] * len(rt_real_after),   # Repeat 'pre' for the same length
        'Type': [case] * len(rt_real_after),  # Repeat 'endo valid' for the same length
        'RT': rt_real_after                        # Reaction times
    })
    df = pd.concat([df, new_rows], ignore_index=True)

    return df

# # Calculate means
# means = [np.mean(rt_sham_before)+50, np.mean(rt_sham_after)+50, np.mean(rt_real_before)+50, np.mean(rt_real_after)+50]

# # Calculate standard errors
# std_errors = [
#     np.std(rt_sham_before) / np.sqrt(len(rt_sham_before)), np.std(rt_sham_after) / np.sqrt(len(rt_sham_after)),
#     np.std(rt_real_before) / np.sqrt(len(rt_real_before)), np.std(rt_real_after) / np.sqrt(len(rt_real_after))
# ]


In [6]:
df_endo_valid = make_tables('endo valid')
df_endo_invalid = make_tables('endo invalid')
df_exo_valid = make_tables('exo valid')
df_exo_invalid = make_tables('exo invalid')

df = pd.concat([df_endo_valid, df_endo_invalid, df_exo_valid, df_exo_invalid], ignore_index=True)

c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\attention_tES\tACS-Attention\processing\behavior\func4behav.py:158: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  behavior_compare = pd.concat([behavior_compare, new_row], ignore_index=True)
C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_7828\1204870281.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_rows], ignore_index=True)
c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Docu

### 3-way ANOVA

In [79]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Assuming df is your DataFrame
# Ensure correct data types
df['Group'] = df['Group'].astype('category')
df['PrePost'] = df['PrePost'].astype('category')
df['Type'] = df['Type'].astype('category')

# Fit the model
model = ols('RT ~ C(Group) * C(PrePost) * C(Type)', data=df).fit()

# Perform ANOVA
anova_results = anova_lm(model, typ=2)  # Using Type II Sum of Squares

# drop residuals row
# anova_results = anova_results.drop('Residual')

# Calculate Eta Squared
anova_results['eta_sq'] = anova_results['sum_sq'] / sum(anova_results['sum_sq'])

# Display the results including F-values, p-values, and eta squared values
anova_results


,sum_sq,df,F,PR(>F),eta_sq
C(Group),1.149205e+05,1.0,18.444859,1.789798e-05,0.004040
C(PrePost),3.695534e+05,1.0,59.313692,1.678031e-14,0.012992
C(Type),2.424289e+06,3.0,129.700288,3.108778e-80,0.085226
C(Group):C(PrePost),1.382431e+05,1.0,22.188148,2.554572e-06,0.004860
C(Group):C(Type),2.014343e+04,3.0,1.077680,3.571980e-01,0.000708
C(PrePost):C(Type),4.900035e+04,3.0,2.621535,4.904014e-02,0.001723
C(Group):C(PrePost):C(Type),2.711406e+04,3.0,1.450611,2.260630e-01,0.000953
Residual,2.530202e+07,4061.0,NaN,NaN,0.889498


In [55]:
import scipy.stats as stats

# Example values: Replace these with your actual F-statistic and degrees of freedom
F_statistic = 5.449566  # This is the F-statistic you obtained from your ANOVA
df1 = 1             # Degrees of freedom between groups (numerator)
df2 = 16           # Degrees of freedom within groups (denominator)

# Calculate the p-value
p_value = 1 - stats.f.cdf(F_statistic, df1, df2)

print("P-value:", p_value)

P-value: 0.0329337744937106


### 2-way ANOVA

#### Endo-Valid

In [80]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Assuming df is your DataFrame
# Ensure correct data types
df = df_endo_valid.drop('Type', axis=1)

df['Group'] = df['Group'].astype('category')
df['PrePost'] = df['PrePost'].astype('category')

# Fit the model
model = ols('RT ~ C(Group) * C(PrePost)', data=df).fit()

# Perform ANOVA
anova_results = anova_lm(model, typ=2)  # Using Type II Sum of Squares

# drop residuals row
# anova_results = anova_results.drop('Residual')

# Calculate Eta Squared
anova_results['eta_sq'] = anova_results['sum_sq'] / sum(anova_results['sum_sq'])

# Display the results including F-values, p-values, and eta squared values
anova_results


,sum_sq,df,F,PR(>F),eta_sq
C(Group),3.740414e+04,1.0,8.464937,0.003669,0.005047
C(PrePost),8.147135e+04,1.0,18.437796,0.000019,0.010992
C(Group):C(PrePost),2.408007e+04,1.0,5.449566,0.019693,0.003249
Residual,7.268785e+06,1645.0,NaN,NaN,0.980712


#### Endo-Invalid

In [88]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Assuming df is your DataFrame
# Ensure correct data types
df = df_endo_invalid.drop('Type', axis=1)

df['Group'] = df['Group'].astype('category')
df['PrePost'] = df['PrePost'].astype('category')

# Fit the model
model = ols('RT ~ C(Group) * C(PrePost)', data=df).fit()

# Perform ANOVA
anova_results = anova_lm(model, typ=2)  # Using Type II Sum of Squares

# drop residuals row
# anova_results = anova_results.drop('Residual')

# Calculate Eta Squared
anova_results['eta_sq'] = anova_results['sum_sq'] / sum(anova_results['sum_sq'])

# Display the results including F-values, p-values, and eta squared values
anova_results

,sum_sq,df,F,PR(>F),eta_sq
C(Group),6.413250e+04,1.0,6.422789,0.011365,0.004096
C(PrePost),1.267410e+05,1.0,12.692950,0.000378,0.008095
C(Group):C(PrePost),4.904192e+04,1.0,4.911487,0.026824,0.003132
Residual,1.541707e+07,1544.0,NaN,NaN,0.984677


In [67]:
import scipy.stats as stats

# Example values: Replace these with your actual F-statistic and degrees of freedom
F_statistic = 4.911487  # This is the F-statistic you obtained from your ANOVA
df1 = 1             # Degrees of freedom between groups (numerator)
df2 = 16           # Degrees of freedom within groups (denominator)

# Calculate the p-value
p_value = 1 - stats.f.cdf(F_statistic, df1, df2)

print("P-value:", p_value)

P-value: 0.041520758276635816


#### Exo-Valid

In [82]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Assuming df is your DataFrame
# Ensure correct data types
df = df_exo_valid.drop('Type', axis=1)

df['Group'] = df['Group'].astype('category')
df['PrePost'] = df['PrePost'].astype('category')

# Fit the model
model = ols('RT ~ C(Group) * C(PrePost)', data=df).fit()

# Perform ANOVA
anova_results = anova_lm(model, typ=2)  # Using Type II Sum of Squares

# drop residuals row
# anova_results = anova_results.drop('Residual')

# Calculate Eta Squared
anova_results['eta_sq'] = anova_results['sum_sq'] / sum(anova_results['sum_sq'])

# Display the results including F-values, p-values, and eta squared values
anova_results

,sum_sq,df,F,PR(>F),eta_sq
C(Group),8.127682e+04,1.0,11.138729,8.644933e-04,0.006562
C(PrePost),2.977254e+05,1.0,40.802312,2.192589e-10,0.024038
C(Group):C(PrePost),1.272453e+05,1.0,17.438562,3.124556e-05,0.010274
Residual,1.187915e+07,1628.0,NaN,NaN,0.959125


In [71]:
import scipy.stats as stats

# Example values: Replace these with your actual F-statistic and degrees of freedom
F_statistic = 17.438562  # This is the F-statistic you obtained from your ANOVA
df1 = 1             # Degrees of freedom between groups (numerator)
df2 = 16           # Degrees of freedom within groups (denominator)

# Calculate the p-value
p_value = 1 - stats.f.cdf(F_statistic, df1, df2)

print("P-value:", p_value)

P-value: 0.0007135010266625663


#### Exo-Invalid

In [83]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Assuming df is your DataFrame
# Ensure correct data types
df = df_exo_invalid.drop('Type', axis=1)

df['Group'] = df['Group'].astype('category')
df['PrePost'] = df['PrePost'].astype('category')

# Fit the model
model = ols('RT ~ C(Group) * C(PrePost)', data=df).fit()

# Perform ANOVA
anova_results = anova_lm(model, typ=2)  # Using Type II Sum of Squares

# drop residuals row
# anova_results = anova_results.drop('Residual')

# Calculate Eta Squared
anova_results['eta_sq'] = anova_results['sum_sq'] / sum(anova_results['sum_sq'])

# Display the results including F-values, p-values, and eta squared values
anova_results

,sum_sq,df,F,PR(>F),eta_sq
C(Group),1.130532e+02,1.0,0.019909,0.887862,0.000049
C(PrePost),6.912337e+03,1.0,1.217267,0.270553,0.002994
C(Group):C(PrePost),1.771266e+03,1.0,0.311921,0.576812,0.000767
Residual,2.299821e+06,405.0,NaN,NaN,0.996190


In [75]:
import scipy.stats as stats

# Example values: Replace these with your actual F-statistic and degrees of freedom
F_statistic = 0.311921  # This is the F-statistic you obtained from your ANOVA
df1 = 1             # Degrees of freedom between groups (numerator)
df2 = 16           # Degrees of freedom within groups (denominator)

# Calculate the p-value
p_value = 1 - stats.f.cdf(F_statistic, df1, df2)

print("P-value:", p_value)

P-value: 0.5842358372834251
